In [2]:
import json
from typing import Optional, List, Dict, Union, Any
from datetime import datetime
import os
import re
import numpy as np
import faiss

class GlobalAIAssistant:
    def __init__(
        self,
        llm_provider: str = "openai",  # openai, anthropic, cohere, or mistral
        embedding_provider: str = "openai",  # openai, cohere, mistral, or voyage
        llm_config: Dict[str, Any] = None,
        embedding_config: Dict[str, Any] = None,
        rerank_config: Dict[str, Any] = None,
        system_prompt: str = "You are a helpful assistant.",
        context_window: int = 5,
        similar_chunks: int = 3,
        rerank_top_k: int = 3,
        verbose: bool = True,
        show_context: bool = False
    ):
        self.llm_provider = llm_provider.lower()
        self.embedding_provider = embedding_provider.lower()
        self.context_window = context_window
        self.similar_chunks = similar_chunks
        self.rerank_top_k = rerank_top_k
        self.verbose = verbose
        self.show_context = show_context
        
        # Initialize components
        self.llm = self._initialize_llm(llm_config or {})
        self.embedding_model = self._initialize_embedding(embedding_config or {})
        self.reranker = self._initialize_reranker(rerank_config or {})
        self.chatbot = self._initialize_chatbot(system_prompt)
        
        # Context markers
        self.context_start_marker = "### Relevant Context ###\n"
        self.context_end_marker = "\n### End Context ###\n"
        self.question_marker = "\nQuestion: "

    def _initialize_llm(self, config: Dict[str, Any]):
        """Initialize the LLM based on provider"""
        if self.llm_provider == "openai":
            from openai import OpenAI_LLM
            return OpenAI_LLM(**config)
        elif self.llm_provider == "anthropic":
            from anthropic import Anthropic_LLM
            return Anthropic_LLM(**config)
        elif self.llm_provider == "cohere":
            from cohere import Cohere_LLM
            return Cohere_LLM(**config)
        elif self.llm_provider == "mistral":
            from mistral import Mistral_LLM
            return Mistral_LLM(**config)
        else:
            raise ValueError(f"Unsupported LLM provider: {self.llm_provider}")

    def _initialize_embedding(self, config: Dict[str, Any]):
        """Initialize the embedding model based on provider"""
        if self.embedding_provider == "openai":
            from openai_embedding import OpenAI_Embedding
            return OpenAI_Embedding(**config)
        elif self.embedding_provider == "cohere":
            from cohere_embedding import Cohere_Embedding
            return Cohere_Embedding(**config)
        elif self.embedding_provider == "mistral":
            from mistral_embedding import Mistral_Embedding
            return Mistral_Embedding(**config)
        elif self.embedding_provider == "voyage":
            from voyage_embedding import Voyage_Embedding
            return Voyage_Embedding(**config)
        else:
            raise ValueError(f"Unsupported embedding provider: {self.embedding_provider}")

    def _initialize_reranker(self, config: Dict[str, Any]):
        """Initialize the Voyage reranker"""
        from voyage_rerank import Voyage_Rerank
        return Voyage_Rerank(top_k=self.rerank_top_k, **config)

    def _initialize_chatbot(self, system_prompt: str):
        """Initialize the chatbot based on LLM provider"""
        if self.llm_provider == "openai":
            from openai import OpenAI_Chatbot
            return OpenAI_Chatbot(self.llm, system_prompt=system_prompt, verbose=self.verbose)
        elif self.llm_provider == "anthropic":
            from anthropic import Anthropic_Chatbot
            return Anthropic_Chatbot(self.llm, system_prompt=system_prompt, verbose=self.verbose)
        elif self.llm_provider == "cohere":
            from cohere import Cohere_Chatbot
            return Cohere_Chatbot(self.llm, system_prompt=system_prompt, verbose=self.verbose)
        elif self.llm_provider == "mistral":
            from mistral import Mistral_Chatbot
            return Mistral_Chatbot(self.llm, system_prompt=system_prompt, verbose=self.verbose)

    def _get_conversation_history(self) -> List[Dict]:
        """Get recent conversation history without retrieval context"""
        history = []
        for msg in self.chatbot.history[-self.context_window:]:
            if msg.get("role") in ["user", "assistant"]:
                content = msg.get("content", "")
                
                # Handle different message formats
                if isinstance(content, list):
                    # Handle OpenAI format
                    content = " ".join([item.get("text", "") for item in content if item.get("type") == "text"])
                elif isinstance(content, dict):
                    # Handle Cohere format
                    content = content.get("text", "")
                
                # Remove context from user messages
                if msg["role"] == "user":
                    content = self._remove_context_from_message(content)
                
                history.append({"role": msg["role"], "content": content})
        return history

    def _remove_context_from_message(self, message: str) -> str:
        """Remove context section from message"""
        if self.context_start_marker in message and self.context_end_marker in message:
            pattern = f"{self.context_start_marker}.*?{self.context_end_marker}"
            message = re.sub(pattern, "", message, flags=re.DOTALL)
            # Remove Question marker if present
            message = message.replace(self.question_marker, "")
        return message.strip()

    def _get_similar_chunks(self, query: str, index_name: str) -> List[str]:
        """Get similar chunks from the embedding index and rerank them"""
        try:
            # Get initial results from embedding search
            results = self.embedding_model.search(index_name, query, k=self.similar_chunks)
            initial_chunks = [result["text"] for result in results]
            
            if not initial_chunks:
                return []
            
            # Rerank the chunks
            if self.verbose:
                print("Reranking retrieved chunks...")
            
            reranked_chunks = self.reranker.get_best_chunks(query, initial_chunks)
            
            if self.verbose:
                print(f"Selected {len(reranked_chunks)} best chunks after reranking")
            
            return reranked_chunks
            
        except Exception as e:
            if self.verbose:
                print(f"Warning: Could not retrieve or rerank chunks: {e}")
            return []

    def _format_message_with_context(self, message: str, chunks: List[str]) -> str:
        """Format message with retrieval context"""
        if not chunks:
            return message
            
        context = "\n".join(f"- {chunk}" for chunk in chunks)
        return (
            f"{self.context_start_marker}"
            f"{context}"
            f"{self.context_end_marker}"
            f"{self.question_marker}{message}"
        )

    def create_knowledge_base(self, texts: List[str], index_name: str):
        """Create a knowledge base from texts"""
        try:
            self.embedding_model.create_faiss_index(index_name, texts)
            if self.verbose:
                print(f"Successfully created knowledge base: {index_name}")
        except Exception as e:
            raise Exception(f"Failed to create knowledge base: {e}")

    def update_knowledge_base(self, new_texts: List[str], index_name: str):
        """Update existing knowledge base with new texts"""
        try:
            self.embedding_model.update_index(index_name, new_texts)
            if self.verbose:
                print(f"Successfully updated knowledge base: {index_name}")
        except Exception as e:
            raise Exception(f"Failed to update knowledge base: {e}")

    def load_existing_knowledge_base(
        self,
        chunks_file: str,
        embeddings_file: str,
        index_name: str
    ) -> None:
        """
        Load an existing knowledge base from chunks.json and embeddings.npy files
        
        Args:
            chunks_file: Path to the chunks JSON file
            embeddings_file: Path to the embeddings NPY file
            index_name: Name to give to the loaded index
        """
        try:
            # Load chunks and metadata
            with open(chunks_file, 'r', encoding='utf-8') as f:
                chunks_data = json.load(f)
                
            # Load embeddings
            embeddings = np.load(embeddings_file)
            
            # Extract texts and create metadata
            texts = [chunk["text"] for chunk in chunks_data]
            chunks_metadata = {
                "created_at": datetime.now().isoformat(),
                "model": self.embedding_model.model,
                "total_chunks": len(texts),
                "embedding_dim": embeddings.shape[1],
                "chunks": [
                    {
                        "id": i,
                        "text": chunk["text"],
                        "embedding_index": i,
                        "original_metadata": chunk.get("metadata", {})
                    }
                    for i, chunk in enumerate(chunks_data)
                ]
            }
            
            # Create FAISS index
            index = faiss.IndexFlatL2(embeddings.shape[1])
            index.add(embeddings)
            
            # Save everything using existing methods
            self.embedding_model.save_index(name=index_name, 
                                          index=index, 
                                          chunks_metadata=chunks_metadata, 
                                          embeddings=embeddings)
            
            if self.verbose:
                print(f"Successfully loaded knowledge base '{index_name}' from files:")
                print(f"- Chunks: {chunks_file}")
                print(f"- Embeddings: {embeddings_file}")
                print(f"- Total chunks: {len(texts)}")
                print(f"- Embedding dimension: {embeddings.shape[1]}")
                
        except Exception as e:
            raise Exception(f"Failed to load knowledge base from files: {e}")

    def chat(self, message: str, index_name: Optional[str] = None) -> str:
        """Process user message with context management"""
        original_message = message  # Keep a copy of the original message
        
        # Get similar chunks if index_name is provided
        if index_name:
            # Get clean conversation history
            history = self._get_conversation_history()
            history_text = "\n".join([f"{msg['role']}: {msg['content']}" for msg in history])
            
            # Combine history and current message for context search
            query = f"{history_text}\n{message}" if history_text else message
            
            # Get new retrieval context with reranking
            similar_chunks = self._get_similar_chunks(query, index_name)
            
            # Format message with new context
            enhanced_message = self._format_message_with_context(message, similar_chunks)
        else:
            enhanced_message = message
        
        # Get response from chatbot
        response = self.chatbot(enhanced_message)
        
        # If show_context is False, replace the enhanced message with the original message in history
        if not self.show_context:
            for msg in self.chatbot.history:
                if msg["role"] == "user" and self.context_start_marker in msg.get("content", ""):
                    if isinstance(msg["content"], str):
                        msg["content"] = original_message
                    elif isinstance(msg["content"], list):  # For OpenAI format
                        msg["content"] = [{"type": "text", "text": original_message}]
                    elif isinstance(msg["content"], dict):  # For Cohere format
                        msg["content"]["text"] = original_message
        
        return response

    def start_new_conversation(self):
        """Start a new conversation"""
        self.chatbot.start_new_conversation()

    def load_conversation(self, conversation_id: str):
        """Load a specific conversation"""
        self.chatbot.load_conversation(conversation_id)

    def list_conversations(self) -> List[str]:
        """List all conversations"""
        return self.chatbot.list_conversations()




In [3]:
llm_config = {
        "model": "gpt-4o-mini", 
        "temperature": 0,
        "max_tokens": 15000,
        "stream": True
    }


In [4]:
embedding_config = {
        "model": "text-embedding-3-large"  # Doit correspondre au modèle utilisé pour créer les embeddings
    }

In [5]:
rerank_config = {
        "model": "rerank-2"
    }


In [7]:
assistant = GlobalAIAssistant(
        llm_provider="openai",
        embedding_provider="openai",  # Doit correspondre au provider utilisé pour créer les embeddings
        llm_config=llm_config,
        embedding_config=embedding_config,
        rerank_config=rerank_config,
        system_prompt="En tant qu’assistant virtuel pour Ouellet Canada, votre mission principale est de fournir des informations précises et fiables sur les produits, services et thématiques autorisées de l’entreprise. *Il est essentiel de ne jamais indiquer que le contexte ne contient pas d’informations. Demande à l’utilisateur de clarifier sur le type de produit ou information souhaitez. Ne dit jamais je ne sais pas où je n’ai pas l’info. Pour les produit ou les informations utilisé du context , incluez des liens directs vers les informations sur le site d’Ouellet Canada.  Assurez-vous d’utiliser systématiquement cette structure d’URL pour garantir l’accès direct à l’information pertinente. Si l'utilisateur pose une question en français, répondez en français. Si l'utilisateur pose une question en anglais, répondez en anglais.  Utilisez les informations du contexte de récupération pour affiner la réponse, mais assurez-vous de répondre dans la langue dans laquelle la question a été posée.",
        context_window=2,
        similar_chunks=150,
        rerank_top_k=20,
        verbose=True,
        show_context=False
    )

In [8]:
assistant.load_existing_knowledge_base(
        chunks_file="/Users/simon-pierreboucher/Downloads/LLM-CHATBOT-main-3/chunks.json",
        embeddings_file="/Users/simon-pierreboucher/Downloads/LLM-CHATBOT-main-3/embeddings.npy",
        index_name="ouellet"
    )

Successfully loaded knowledge base 'ouellet' from files:
- Chunks: /Users/simon-pierreboucher/Downloads/LLM-CHATBOT-main-3/chunks.json
- Embeddings: /Users/simon-pierreboucher/Downloads/LLM-CHATBOT-main-3/embeddings.npy
- Total chunks: 1656
- Embedding dimension: 3072


In [9]:
assistant.chat(
        "Quel est l'identifiant exacte du Kit de terminaison et de raccord d'alimentation avec étiquette d'avertissement Série HA pour produit Kit de terminaison et de raccord d'alimentation avec étiquette d'avertissement OSR",
        index_name="ouellet"
    )

Reranking retrieved chunks...
Selected 20 best chunks after reranking

chatbot_2 - User:  ### Relevant Context ###
- # Catalogue de Produits 2024  Produit : ELVB-SREX-M20-HT1 - **Prix** : 192.00 $ - **Description** : Raccord d’alimentation avec connecteur plaqué de nickel, M20, emplacements dangereux #### Produit : KIT-OSR-ELSR-HA - **Prix** : 92.00 $ - **Description** : Kit de terminaison et de raccord d'alimentation avec étiquette d'avertissement Série HA ## Remarques 1. Température minimale d’installation : -45 °C (-49 °F). 2. Température minimale d’installation : -13 °C (-25 °F). 3. L’identification du type de tuyaux est le facteur clé à un raccordement simple et efficace, Ouellet Canada n’est pas responsable d’une mauvaise sélection de raccord Philmac. Aucun retour ne sera accepté.
- # Catalogue de Produits 2024 .00 $ - **Description** : Trousse de raccord de plomberie rapide pour tuyau de polyéthylène de 1” ID pour câble MA-BF #### Produit : KIT-OSR-MABF-PH-114-ID3 - **Prix** : 1

"L'identifiant exact du Kit de terminaison et de raccord d'alimentation avec étiquette d'avertissement Série HA est **KIT-OSR-ELSR-HA**. \n\nPour plus d'informations, vous pouvez consulter la page produit sur le site d'Ouellet Canada [ici](https://www.ouellet.com/fr-ca/produits/osr-ha.aspx?aliaspath=%2fProduits%2fOSR-HA)."

In [10]:
assistant.chat(
        "Donne moi les détaille de la série OCB , PRIX ET DIMENSION",
        index_name="ouellet"
    )

Reranking retrieved chunks...
Selected 20 best chunks after reranking

chatbot_2 - User:  ### Relevant Context ###
- # Catalogue des Produits OSR  **ECA-JB2** : Boîte de jonction pour connexion double avec étiquette et fixations. - **EL-ECH-EX1** : Terminaison d'extrémité pour ELSR-HA. - **ELAK-3-7** : Boîte de jonction avec support de montage pour emplacements non dangereux (ordinaires) et dangereux. - **ECA-LSTK** : Kit de connexion d'alimentation et de terminaison éclairée. - **ELVB-SRAH** : Connexion d'alimentation sans presse-étoupe. - **ELVB-SRAH-34-ST** : Connexion d'alimentation avec presse-étoupe en acier/zinc, 3/4'' NPT, emplacement non dangereux. - **ELVB-SREX-34-HT1** : Connexion d'alimentation avec presse-étoupe nickelé, 3/4'' NPT, emplacement dangereux. - **ELVB-SREX-M20-HT1** : Connexion d'alimentation avec presse-étoupe nickelé, M20, emplacement dangereux. - **KIT-OSR-ELSR-HA** : Kit de terminaison d'extrémité et d'alimentation avec autocollant d'avertissement HA Series

"Voici les détails de la série OCB, y compris les prix et les dimensions des produits disponibles :\n\n### Produits de la série OCB\n\n1. **Produit : OCB070X-1925**\n   - **Watts** : 150\n   - **Prix** : 851.00 $\n   - **Longueur** : 75 13/16 po / 1925 mm\n   - **Poids** : 21.5 lb / 9.8 kg\n\n2. **Produit : OCB105X-1925**\n   - **Watts** : 150\n   - **Prix** : Non spécifié\n   - **Longueur** : Non spécifié\n   - **Poids** : Non spécifié\n\n3. **Produit : OCB140X-1925**\n   - **Watts** : 200\n   - **Prix** : Non spécifié\n   - **Longueur** : Non spécifié\n   - **Poids** : Non spécifié\n\n4. **Produit : OCB080X-2148**\n   - **Watts** : 150\n   - **Prix** : 914.00 $\n   - **Longueur** : 84 9/16 po / 2148 mm\n   - **Poids** : 24.0 lb / 10.9 kg\n\n5. **Produit : OCB120X-2148**\n   - **Watts** : 200\n   - **Prix** : Non spécifié\n   - **Longueur** : Non spécifié\n   - **Poids** : Non spécifié\n\n6. **Produit : OCB160X-2148**\n   - **Watts** : 200\n   - **Prix** : Non spécifié\n   - **Longueu

In [11]:
assistant.chat(
        "Peux tu me donner le guide installation et celui de conception pour le produit OSR-MA ? ",
        index_name="ouellet"
    )

Reranking retrieved chunks...
Selected 20 best chunks after reranking

chatbot_2 - User:  ### Relevant Context ###
- # Catalogue de Produits OCB  / 2606 mm - **Poids** : 29.1 lb / 13.2 kg ## Modèles à Basse Densité Moyenne ### Produit : OCB020X-711 - **Watts** : 100 - **Prix** : Non spécifié - **Longueur** : 28 po / 711 mm - **Poids** : Non spécifié ### Produit : OCB030X-711 - **Watts** : 150 - **Prix** : 597.00 $ - **Longueur** : 28 po / 711 mm - **Poids** : 8.0 lb / 3.6 kg ### Produit : OCB040X-711 - **Watts** : 200 - **Prix** : Non spécifié - **Longueur** : Non spécifié - **Poids** : Non spécifié ### Produit : OCB030X-965 - **Watts** : 150 - **Prix** : 641.00 $ - **Longueur** : 38 po / 965 mm - **Poids** : 10.8 lb / 4.9 kg ### Produit : OCB045X-965 - **Watts** : 150 - **Prix** : Non spécifié - **Longueur** : Non spécifié - **Poids** : Non spécifié ### Produit : OCB060X-965 - **Watts** : 200 - **Prix** : Non spécifié - **Longueur** : Non spécifié - **Poids** : Non spécifié ### Produi

"Pour obtenir le guide d'installation et le guide de conception pour le produit OSR-MA, vous pouvez consulter les documents techniques disponibles sur le site d'Ouellet Canada. Voici les liens directs vers ces documents :\n\n- **Guide d'installation pour OSR-MA** : [Guide d'installation OSR-MA](https://www.ouellet.com/getattachment/Produits/OSR-MA/OSR-HA_Manuel-d-utilisation.pdf.aspx?lang=fr-CA)\n- **Guide de conception pour OSR-MA** : [Guide de conception OSR-MA](https://www.ouellet.com/getattachment/Produits/OSR-MA/GUIDE-OSR-F.pdf.aspx?lang=fr-CA)\n\nCes documents contiennent des informations détaillées sur l'installation et la conception des produits de la série OSR-MA. Si vous avez besoin d'autres informations, n'hésitez pas à demander !"

In [12]:
assistant.chat(
        "Quel est l'adresse du siege social de Ouellet ? ",
        index_name="ouellet"
    )

Reranking retrieved chunks...
Selected 20 best chunks after reranking

chatbot_2 - User:  ### Relevant Context ###
- Source: https://www.ouellet.com/fr-ca/produits/osr-ma.aspx?cat=/Produits/Produits-RESIDENTIELS/Cables-chauffants-autoregulants Source: https://www.ouellet.com/fr-ca/produits/osr-ma.aspx?cat=/Produits/Produits-RESIDENTIELS/Cables-chauffants-autoregulants # OSR-MA - [ Options pour câbles chauffants autorégulants Série OSR-Options ](https://www.ouellet.com/fr-ca/produits/osr-options.aspx) Câble chauffant autorégulant micro ELSR-MA Série OSR-MA Protection contre le gel. Traçage interne de conduites ou de tuyaux (BF). Appropriés pour : \- I’industrie de transformation alimentaire; \- les applications de réfrigération; \- les systèmes de gicleurs automatiques. * Très flexibles et sont utilisés pour les installations qui requièrent un câble de plus faible diamètre et des circuits chauffants plus courts. * La version BF du câble est approuvée pour utilisation avec des conduites 

"L'adresse du siège social de Ouellet Canada est la suivante :\n\n**Ouellet Canada inc.**  \n180, 3e Avenue  \nL'Islet (Québec)  \nG0R 2C0  \nCANADA\n\nSi vous avez besoin de plus d'informations, n'hésitez pas à demander !"

In [13]:
assistant.chat(
        "détaille moi le produit OLI",
        index_name="ouellet"
    )

Reranking retrieved chunks...
Selected 20 best chunks after reranking

chatbot_2 - User:  ### Relevant Context ###
- Source: https://www.ouellet.com/Nous-joindre/Contacts.aspx Source: https://www.ouellet.com/Nous-joindre/Contacts.aspx # Contacts - [**Écrivez-nous**](https://www.ouellet.com/fr-ca/fr-ca=) Aidez-nous à vous procurer un service rapide et efficace en prenant soin de fournir vos coordonnées de façon complète et exacte. | Nom:| ---|--- Prénom:| Compagnie:| Téléphone:| Courriel:| Sujet:| Questions techniques Demande de soumission Demande d'échantillons Suivi de production/expédition Demande d'emploi Informations diverses Commentaires:| ---|--- | - [**Coordonnées**](https://www.ouellet.com/fr-ca/fr-ca=) Ouellet Canada inc. 180, 3e Avenue L'Islet (Québec) G0R 2C0 CANADA Sans frais : 1 800 463-7043 Téléphone : 418 247-3947 Télécopieur : 418 247-7801 - [Courriel : [[email protected]](https://www.ouellet.com/cdn-cgi/l/email-protection#6a03040c052a051f0f06060f1e44090507) **Centre de

"Voici les détails du produit **OLI**, qui est un convecteur architectural à haute densité :\n\n### OLI - High Density Architectural Sloped Top Cabinet Convector\n\n#### Caractéristiques :\n- **Type** : Convecteur à dessus incliné\n- **Couleur** :\n  - Standard : blanc, amande\n  - Optionnel (supplément applicable) : argent métallique, bronze, charbon métallique, aluminium, noir semi-brillant, sable\n  - Couleurs personnalisées disponibles sur demande\n- **Densité de puissance** :\n  - Densité de puissance standard moyenne de 1000W/ft\n  - Densité de puissance faible moyenne de 500W/ft, 750W/ft\n- **Contrôle** :\n  - Thermostat intégré à l'épreuve des manipulations disponible\n  - Relais mécaniques, électroniques ou pneumatiques disponibles\n- **Finition** : Peinture époxy/polyester en poudre\n- **Tension** : \n  - 208V, 240V, 277V, 347V, 600V, 1 ou 3 phases\n- **Construction** :\n  - Cabinet en acier de 20-gauge\n  - Couverture frontale en acier de 16-gauge\n  - Grilles en aluminium e